<a href="https://colab.research.google.com/github/jacobsomer/AlphaGoLite/blob/master/AlphaGoLite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [171]:
from tensorflow.keras import Input, Model
from sklearn.datasets import load_iris
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.initializers import glorot_uniform
import numpy as np
from keras.optimizers import *
"""
AlphaGo NeuralNetwork Implementation
Input Shape: (19,19,17)
Policy head shape: (361+1,1)
Value head shape:(1,) --> Values
"""

'\nAlphaGo NeuralNetwork Implementation\nInput Shape: (19,19,17)\nPolicy head shape: (361+1,1)\nValue head shape:(1,) --> Values\n'

In [3]:
data, target = load_iris(return_X_y=True)
X = data[:, (0, 1, 2)]
Y = data[:, 3]
Z = target

inputs = Input(shape=(3,))
x = Dense(16, activation='relu')(inputs)
x = Dense(32, activation='relu')(x)
output1 = Dense(1)(x)
output2 = Dense(3, activation='softmax')(x)

model = Model(inputs=inputs, outputs=[output1, output2])

model.compile(loss=['mae', 'sparse_categorical_crossentropy'], optimizer='adam')

history = model.fit(X, [Y, Z], epochs=10, batch_size=8)

Epoch 1/10
19/19 [==============================] - 1s 2ms/step - loss: 1.7356 - dense_2_loss: 0.5533 - dense_3_loss: 1.1823
Epoch 2/10
19/19 [==============================] - 0s 2ms/step - loss: 1.5182 - dense_2_loss: 0.4566 - dense_3_loss: 1.0617
Epoch 3/10
19/19 [==============================] - 0s 2ms/step - loss: 1.3897 - dense_2_loss: 0.4108 - dense_3_loss: 0.9788
Epoch 4/10
19/19 [==============================] - 0s 2ms/step - loss: 1.2711 - dense_2_loss: 0.3614 - dense_3_loss: 0.9097
Epoch 5/10
19/19 [==============================] - 0s 2ms/step - loss: 1.1638 - dense_2_loss: 0.3011 - dense_3_loss: 0.8627
Epoch 6/10
19/19 [==============================] - 0s 2ms/step - loss: 1.0645 - dense_2_loss: 0.2538 - dense_3_loss: 0.8107
Epoch 7/10
19/19 [==============================] - 0s 1ms/step - loss: 0.9616 - dense_2_loss: 0.1974 - dense_3_loss: 0.7642
Epoch 8/10
19/19 [==============================] - 0s 2ms/step - loss: 0.8830 - dense_2_loss: 0.1644 - dense_3_loss: 0.7186


In [6]:
print(Y.shape)

(150,)


In [76]:
def residual_layer(X, filter_size):
    
    # Save the input value. You'll need this later to add back to the main path.
    X_skip_connection = X

    # First component
    X = Conv2D(filters=256, kernel_size=filter_size, strides=(1, 1),  kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

     # Second component of main path same as first
    X = Conv2D(filters=256, kernel_size=filter_size, strides=(1, 1),  kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    X_skip_connection = Conv2D(filters=256, kernel_size=filter_size, strides=(1, 1),  kernel_initializer=glorot_uniform(seed=0))(X_skip_connection)
    X_skip_connection = Conv2D(filters=256, kernel_size=filter_size, strides=(1, 1),  kernel_initializer=glorot_uniform(seed=0))(X_skip_connection)
    # Final step: Add skip_connection value to main path, and pass it through a RELU activation
    X = Add()([X, X_skip_connection])
    X = Activation('relu')(X)
    
    return X


def convolutional_layer(X):
    # First component
    X = Conv2D(filters=256, kernel_size=(1, 1), strides=(1, 1),  kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    return X

def value_head(X):
    X = Conv2D(filters=1, kernel_size=(1,1), strides=(1, 1),  kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = Dense(256)(X)
    X = Activation('relu')(X)
    X = Dense(1)(X)
    X = Activation('tanh')(X)
    return X

def policy_head(X):
    X = Conv2D(filters=1, kernel_size=(1, 1), strides=(1, 1),  kernel_initializer=glorot_uniform(seed=0))(X)
    X = Conv2D(filters=1, kernel_size=(1, 1), strides=(1, 1),  kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    
    #19x19+1 -> all possible moves + pass
    X = Dense(362)(X)
    return X

In [193]:

def AlphaGo_Zero_Network():
  inputs = Input(shape=(19,19,17))
  # 1 convolutional layer
  X = convolutional_layer(inputs)
  
  # 5 residual layers instead of 40 
    
  X=residual_layer(X, (3,3))
  X=residual_layer(X, (3,3))
  X=residual_layer(X, (3,3))
  X=residual_layer(X, (3,3))
  X=residual_layer(X, (2,2))
  #value head
  value_head_output = value_head(X)

  #policy head
  policy_head_output = policy_head(X)

  X = Model(inputs=inputs, outputs=[value_head_output, policy_head_output])
  
  return X
nn= AlphaGo_Zero_Network()


In [194]:
rsmprop = RMSprop(clipvalue=0.5)
nn.compile(loss=['binary_crossentropy', 'categorical_crossentropy'], optimizer='rmsprop')

In [ ]:
import random
x=[[[[random.choice([1,0]) for x in range(17)] for j in range(19)] for k in range(19)] for t in range(500)]
x = np.array(x, dtype="float32")
y=[[random.choice([1,0])] for x in range(500)]
y = np.array(y, dtype="float32")
z=[[[[random.choice([1,0]) for i in range(362)]]] for x in range(500)]
z = np.array(z, dtype="float32")
# model.predict(x)
history = nn.fit(x, [y, z], epochs=1, batch_size=1)

200/500 [===========>..................] - ETA: 1:11 - loss: 1937.0110 - activation_3227_loss: 7.0909 - dense_75_loss: 1929.9202